# Power BI Semantic Model Connection (Local)

Connect to Power BI Semantic Models from a **local Python environment** using the REST API.

## Prerequisites
1. **Azure CLI authentication**: Run `az login` before starting
2. **Power BI Premium, PPU, or Fabric capacity**: Required for executing DAX queries
3. **Workspace access**: You need at least read access to the workspace and semantic model

> **Note**: The `semantic-link` library's XMLA-based functions (`read_table`, `list_tables`) only work inside Fabric notebooks. This notebook uses the REST API which works locally.

In [ ]:
import pandas as pd
import requests
import json
from azure.identity import DefaultAzureCredential, DeviceCodeCredential
from azure.core.exceptions import ClientAuthenticationError

In [ ]:
# Authenticate to Power BI
try:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
    print("✓ Authenticated using DefaultAzureCredential")
except ClientAuthenticationError:
    print("DefaultAzureCredential failed. Using DeviceCodeCredential...")
    credential = DeviceCodeCredential()
    token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
    print("✓ Authenticated using DeviceCodeCredential")

## Power BI REST API Helper Class

A reusable class for interacting with Power BI semantic models from local Python.

In [ ]:
from powerbi_client import PowerBIClient

# Initialize the client
pbi = PowerBIClient(credential)
print("✓ PowerBIClient initialized")

## 1. List Workspaces and Datasets

In [ ]:
# List workspaces
df_workspaces = pbi.list_workspaces()
print(f"Found {len(df_workspaces)} workspaces")
display(df_workspaces[["name", "id", "isOnDedicatedCapacity"]].head(10))

In [ ]:
# Configuration - set your workspace and dataset names
WORKSPACE_NAME = "perry-pbi-demo-workspace"
DATASET_NAME = "Customer Profitability Sample"

# Check if workspace is on Premium capacity
if pbi.is_premium(WORKSPACE_NAME):
    print(f"✓ Workspace '{WORKSPACE_NAME}' is on Premium/Fabric capacity")
else:
    print(f"✗ Workspace '{WORKSPACE_NAME}' is NOT on Premium capacity - DAX queries will fail!")

# List datasets in workspace
df_datasets = pbi.list_datasets(WORKSPACE_NAME)
print(f"\nDatasets in '{WORKSPACE_NAME}':")
display(df_datasets[["name", "id", "configuredBy"]])

## Describe Dataset - Get Full Schema

In [ ]:
# Test describe_dataset method
result = pbi.describe_dataset(WORKSPACE_NAME, DATASET_NAME)
print(f"Found {len(result['tables'])} tables\n")
print("=" * 60)
for table in result['tables']:
    print(f"\n📊 {table['name']} ({len(table['columns'])} columns)")
    for col in table['columns']:
        card = f"({col['cardinality']} unique)" if col.get('cardinality') else ""
        dtype = f"[{col.get('dataType', '?')}]"
        print(f"   └─ {col['name']} {dtype} {card}")

print("\n" + "=" * 60)
print("\n🔗 Inferred Relationships:")
for rel in result['relationships']:
    print(f"   {rel['keyColumn']}: {' <-> '.join(rel['tables'])}")

print("\n" + "=" * 60)
print("\n📝 LLM Context (first 2000 chars):")
print(result['llm_context'][:2000])

## 2. Read Data from a Table

In [ ]:
# Read the first 10 rows from the 'Customer' table
df_customers = pbi.read_table(WORKSPACE_NAME, DATASET_NAME, "Customer", top_n=10)
print(f"Retrieved {len(df_customers)} rows from 'Customer' table")
display(df_customers)

## 3. Execute Custom DAX Query

In [ ]:
# Execute a custom DAX query
dax_query = """
EVALUATE
SUMMARIZECOLUMNS(
    'Customer'[State],
    "Customer Count", COUNTROWS('Customer')
)
ORDER BY [Customer Count] DESC
"""

df_result = pbi.execute_dax(WORKSPACE_NAME, DATASET_NAME, dax_query)
print(f"Query returned {len(df_result)} rows")
display(df_result)

## 4. Evaluate a Measure

In [ ]:
# Evaluate a measure (replace with an actual measure from your model)
# Example: Evaluate [Total Revenue] grouped by State
try:
    df_measure = pbi.evaluate_measure(
        WORKSPACE_NAME, 
        DATASET_NAME, 
        "[Total Revenue]",  # Replace with your measure name
        group_by=["'Customer'[State]"]
    )
    display(df_measure.head(10))
except Exception as e:
    print(f"Note: {e}")
    print("Tip: Replace '[Total Revenue]' with an actual measure from your semantic model")